# IMPORT LIBRARIES

In [1]:
import cobra
import glob
import os
import time
import pandas as pd

# RECON 3D

In [2]:
recon3D = cobra.io.load_matlab_model('/Users/claudiavicentecomorera/Desktop/TFG/Data/Recon3D_301/Recon3DModel_301.mat')
recon3D

No defined compartments in model Recon3DModel. Compartments will be deduced heuristically using regular expressions.
Using regular expression found the following compartments:c, e, g, i, l, m, n, r, x


Name,Recon3DModel
Memory address,1266acad0
Number of metabolites,5835
Number of reactions,10600
Number of genes,2248
Number of groups,103
Objective expression,1.0*biomass_maintenance - 1.0*biomass_maintenance_reverse_95d2f
Compartments,"c, l, m, r, e, x, n, g, i"


In [ ]:
solution_recon_3D = recon3D.optimize()
bm = solution_recon_3D.objective_value
solution_recon_3D

# FLUXES SIMULATION: rFASTCORMICS EXAMPLE_V4

In [ ]:
model1 =  cobra.io.load_matlab_model("/Users/claudiavicentecomorera/Desktop/TFG/rFASTCORMICS/rFASTCORMICS_RNA-seq/exampleResults/Example_v4_3D/SampleModel_TCGA06067511A32RA36H07.mat")
model2 =  cobra.io.load_matlab_model("/Users/claudiavicentecomorera/Desktop/TFG/rFASTCORMICS/rFASTCORMICS_RNA-seq/exampleResults/Example_v4_3D/SampleModel_TCGA06067811A32RA36H07.mat")

In [ ]:
model1

In [ ]:
model2

In [ ]:
t0_1 = time.time()  
solution1 = model1.optimize()
t_1 = time.time()  
time1 = t_1 - t0_1
print(f"The simulation of {model1.id} fluxes takes: {time1} seconds")

bm1 = solution1.objective_value
solution1

In [ ]:
t0_2 = time.time()  
solution2 = model2.optimize()
t_2 = time.time()  
time2 = t_2 - t0_2
print(f"The simulation of {model2.id} fluxes takes: {time2} seconds")

bm2 = solution2.objective_value
solution2

In [ ]:
print((bm1/bm)*100)
print((bm2/bm)*100)

# FLUXES SIMULATION: rFASTCORMICS ADDITIONAL EXAMPLE

In [ ]:
f_path = "/Users/claudiavicentecomorera/Desktop/TFG/rFASTCORMICS/rFASTCORMICS_RNA-seq/exampleResults/Example_BRCA_3D/"
file_paths = glob.glob(f_path + "*.mat")

models = []
solutions = []
times = []

for file_path in file_paths:
    model = cobra.io.load_matlab_model(file_path)
    model.id = os.path.splitext(os.path.basename(file_path))[0]

    t0_add = time.time()  
    solution = model.optimize()
    t_add = time.time()  
    time_add = t_add - t0_add 
    
    print(f"The simulation of {model.id} fluxes takes: {time_add} seconds")
    
    models.append(model)
    solutions.append(solution)
    times.append(time_add)

In [ ]:
solutions[1]

In [ ]:
r_models = pd.DataFrame(index=[model.id for model in models])

for i, solution in enumerate(solutions, start = 0):
    bm_m = solution.objective_value
    percent = (bm_m/bm) * 100
    
    r_models.loc[r_models.index[i], 'flux_bm'] = bm_m
    r_models.loc[r_models.index[i], 'percent_bm'] = percent
    r_models.loc[r_models.index[i], 'feasible_models'] = "TRUE" if percent > 5 else "FALSE"

print(f'The 5% of Recon 2 v.04 objective value is {5/100*bm}')
print(f'There are {r_models["feasible_models"].eq("FALSE").sum()} models that aren\'t feasible because of their low objective value.\n')
r_models